<a href="https://colab.research.google.com/github/akhilesh008/AIML-Training-Akhilesh/blob/main/English_to_Hindi_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Language_Translation/Hindi_English_Truncated_Corpus.csv', encoding='utf-8')
print(data.head())

      source                                   english_sentence  \
0        ted  politicians do not have permission to do what ...   
1        ted         I'd like to tell you about one such child,   
2  indic2012  This percentage is even greater than the perce...   
3        ted  what we really mean is that they're bad at not...   
4  indic2012  .The ending portion of these Vedas is called U...   

                                      hindi_sentence  
0  राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...  
1  मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...  
2   यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।  
3     हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते  
4        इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।  


## Data Cleaning and preprocessing

In [3]:
pd.isnull(data).sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

#### There are some null values under english_sentences. so remove it and repopulate the data

In [4]:
data = data[~pd.isnull(data['english_sentence'])]
pd.isnull(data).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

#### Remove duplicates

In [5]:
data.drop_duplicates(inplace=True)

### Experimenting with some subset of the data - 25000 samples

In [6]:
data = data.sample(n=25000)
data.shape

(25000, 3)

### Do some preprocessing to the dataset like converting to lowercase , remove quotes, punctuations, numbers . Add start and end token to the dataset

In [7]:
# lowercase conversion
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.lower())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.lower())

In [8]:
# removing quotes
import re
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub("'",'',x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("'",'',x))

In [9]:
# remove punctuations
# get all the punctuations
import string
punc = set(string.punctuation)
data['english_sentence'] = data['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in punc))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if x not in punc))


In [10]:
# remove all numbers
from string import digits
remove_digits = str.maketrans('','',digits)
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

#remove extra space
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.strip())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.strip())
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub(' +'," ",x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub(' +'," ",x))

In [11]:
#add start and end tokens to each target sentence , i.e hindi sentence
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [12]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# print the pre-processed dataframe header

data.head()

<ipython-input-12-bd27ff60aa36>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,source,english_sentence,hindi_sentence
4836,indic2012,jaiselmer,START_ जैसलमेर _END
6116,ted,so in financial services a great deal of opportunity,"START_ तो वित्तीय सेवाओं में भी, अवसर का एक महान सौदा। _END"
123650,tides,when after a prolonged fight the british regained control of delhi bahadur shah went into hiding,START_ एक लंबे संघर्ष के बाद जब अंग्रेजों ने दोबारा दिल्ली पर अधिकार जमा लिया तो तो बहादुरशाह ज़फर किसी गुप्त स्थान में जाकर छिप गये थे . _END
59813,ted,more recently that power is untethering and leaping off the desktops,"START_ आजकल तो मामला डेस्कटॉप से हट कर आगे बढ गया है, _END"
46746,indic2012,why the sky should be made wet by weeping,START_ चर्ख (आसमान) से क्यों ग़िला करें _END


### Get english and Hindi vocab

In [13]:
all_eng_words = set()
for eng in data['english_sentence']:
  for word in eng.split():
    if word not in all_eng_words:
      all_eng_words.add(word)

all_hindi_words = set()
for hindi in data['hindi_sentence']:
  for word in hindi.split():
    if word not in all_hindi_words:
      all_hindi_words.add(word)

print('English vocab length - {}'.format(len(all_eng_words)))
print('Hindi Vocab length - {}'.format(len(all_hindi_words)))

English vocab length - 30596
Hindi Vocab length - 39473


### Add length of each sentence as column to the dataset

In [14]:
data['length_english_sentence'] = data['english_sentence'].apply(lambda x:len(x.split(" ")))
data['length_hindi_sentence'] = data['hindi_sentence'].apply(lambda x:len(x.split(" ")))

data.head()

,source,english_sentence,hindi_sentence,length_english_sentence,length_hindi_sentence
4836,indic2012,jaiselmer,START_ जैसलमेर _END,1,3
6116,ted,so in financial services a great deal of opportunity,"START_ तो वित्तीय सेवाओं में भी, अवसर का एक महान सौदा। _END",9,12
123650,tides,when after a prolonged fight the british regained control of delhi bahadur shah went into hiding,START_ एक लंबे संघर्ष के बाद जब अंग्रेजों ने दोबारा दिल्ली पर अधिकार जमा लिया तो तो बहादुरशाह ज़फर किसी गुप्त स्थान में जाकर छिप गये थे . _END,16,29
59813,ted,more recently that power is untethering and leaping off the desktops,"START_ आजकल तो मामला डेस्कटॉप से हट कर आगे बढ गया है, _END",11,13
46746,indic2012,why the sky should be made wet by weeping,START_ चर्ख (आसमान) से क्यों ग़िला करें _END,9,8


In [15]:
data[data['length_english_sentence']>30].shape

(2411, 5)

In [16]:
data = data[data['length_english_sentence']<=20]
data = data[data['length_hindi_sentence']<=20]

In [17]:
data.shape

(16032, 5)

### Store the maximum length of english and hindi sentence

In [18]:
max_eng_sentence = max(data['length_english_sentence'])
max_hin_sentence = max(data['length_english_sentence'])

print('Max lenth of english sentence - {}'.format(max_eng_sentence))
print('Max length of hindi senetce - {}'.format(max_hin_sentence))

Max lenth of english sentence - 20
Max length of hindi senetce - 20


### Get the english and hindi words in sorted order
### Also get the encoder and decoder tokens

In [19]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_token = len(all_eng_words)
num_decoder_token = len(all_hindi_words)

print('Encoder tokens {} ; Decoder Tokens {}'.format(num_encoder_token, num_encoder_token))

Encoder tokens 30596 ; Decoder Tokens 30596


### add one extra token for zero padding

In [20]:
num_decoder_token +=1

In [21]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,source,english_sentence,hindi_sentence,length_english_sentence,length_hindi_sentence
57881,ted,were very afraid,START_ हम बहुत डरे हुए हैं | _END,3,8
60199,ted,for the elite administrators laughter who kept track of accounts for the empires,START_ उन संभांत आकाओं के लिये (हँसी) जो साम्राज्यों का हिसाब रखते थे। _END,13,14
106303,ted,around dick feynman,START_ वो भी दिक् फेय्न्मन की सामने. _END,3,8
28457,ted,historically for example,START_ ऐतिहासिक रूप से _END,3,5
63815,tides,thereafter even that avenue was closed,START_ उसके बाद वह भी दरवाजा बंद हो गया . _END,6,11


### Split the data in train and test

In [22]:
from sklearn.model_selection import train_test_split

X, y = data['english_sentence'], data['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
print('Train size - {}; Test Size - {}'.format(X_train.shape, X_test.shape))

Train size - (12825,); Test Size - (3207,)


### Save datasets

In [23]:
X_train.to_pickle('/content/drive/MyDrive/Language_Translation/X_train.pkl')
X_test.to_pickle('/content/drive/MyDrive/Language_Translation/X_test.pkl')

### Generate training and testing batch

In [24]:
import numpy as np

def generate_batch(X=X_train, y=y_train, batch_size=128):
  while True:
    for j in range(0, len(X), batch_size):
      encoder_input_data = np.zeros((batch_size, max_eng_sentence),dtype='float32')
      decoder_input_data = np.zeros((batch_size,max_hin_sentence),dtype='floar32')
      decoder_target_data = np.zeros((batch_size, max_hin_sentence, num_decoder_token),dtype='float32')

      for i, (input_text, output_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
        for t, word in enumerate(input_text):
          encoder_input_data[i,t] = input_token_index[word]
        for t, word in enumerate(output_text):
          if t<len(output_text.split())-1:
            decoder_input_data[i,t] = target_token_index[word]
          if t>0:
            # decoder target sequence (one hot encoded)
            # does not include the START_ token
            # Offset by one timestep
            decoder_target_data[i, t - 1, target_token_index[word]] = 1
      yield([encoder_input_data, decoder_input_data],decoder_target_data)